# Exploring math and ELA proficiencies

We will explore whether trends emerge in math and ELA proficiencies, based on the clustering of schools. We are looking at `% Level 3+4`, which would show the percentage of students who meet learning standards (with/without distinction).

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("output/clustered-2024.csv")

In [3]:
cols_to_use = [ "DBN", "School Name", "Grade", "Year", "Number Tested", "% Level 3+4" ]
math_df = pd.read_excel("data/school-math-results-2018-2025-public.xlsx", sheet_name="Math - All", usecols=cols_to_use)
ela_df = pd.read_excel("data/school-ela-results-2018-2025-public.xlsx", sheet_name="ELA - All", usecols=cols_to_use)

In [4]:
merged_df = df.copy()

In [5]:
test_data = {
    "math": (math_df, "math_pct"),
    "ela": (ela_df, "ela_pct")
}

for test_name, (test_df, col_name) in test_data.items():
    # filter all grades and latest year
    filtered = test_df[(test_df["Grade"] == "All Grades") & (test_df["Year"] == 2025)]

    # merge and rename cols
    merged_df = merged_df.merge(
        filtered[["DBN", "% Level 3+4"]].rename(columns={"% Level 3+4": col_name}),
        on="DBN", how="left"
    )

In [6]:
merged_df.sample(3)

,DBN,School Name,Year,% Asian and Pacific Islander,% Black,% Hispanic,% Multi-Racial,% Native American,% White,% Missing Race/Ethnicity Data,Borough,kmeans_label,math_pct,ela_pct
593,11X545,Bronx Aerospace High School,2024-25,0.021622,0.383784,0.518919,0.010811,0.024324,0.029730,0.010811,X,1,NaN,NaN
1054,22K361,P.S. 361 East Flatbush Early Childhood School,2024-25,0.059278,0.662371,0.229381,0.012887,0.010309,0.025773,0.000000,K,3,40.384617,59.223301
614,12X248,"Metropolitan High School, The",2024-25,0.000000,0.372881,0.593220,0.022599,0.005650,0.005650,0.000000,X,1,NaN,NaN


In [7]:
merged_df.to_csv("output/test-scores.csv", encoding="UTF-8", index=False)

In [8]:
merged_df[["math_pct", "ela_pct"]] = merged_df[["math_pct", "ela_pct"]].apply(pd.to_numeric, errors="coerce")

*Note*: Data caveat - `math_pct` and `ela_pct` now includes suppressed or null data.

In [9]:
math_trends = merged_df.groupby("kmeans_label")["math_pct"].agg(["mean", "min", "max", "std"])
math_trends.T

kmeans_label,0,1,2,3,4
mean,72.514923,42.917546,43.888176,49.679284,72.555059
min,37.755100,7.228916,5.932203,12.820513,15.384615
max,99.404762,95.973152,97.368423,98.529411,100.000000
std,13.801632,18.235831,15.791776,19.153974,15.356550


In [10]:
ela_trends = merged_df.groupby("kmeans_label")["ela_pct"].agg(["mean", "min", "max", "std"])
ela_trends.T

kmeans_label,0,1,2,3,4
mean,68.082826,45.292373,43.048934,51.409843,72.749463
min,39.917694,8.771930,16.410257,22.602739,29.166666
max,98.802399,98.924728,94.174759,96.480942,100.000000
std,13.082734,15.703252,13.275849,15.930233,13.593726


## Observations

Generally: 
Cluster 4 is top performer in both math and ELA (math: 72.6%, ela: 72.7%), with Cluster 0 as second best (math: 72.5%, ela: 68.1%). Clusters 1-3 are showing swings in math proficiency rates.

* Less variability in ELA than math proficiencies 
* For clusters serving similar populations, Clusters 1-3 are getting a wide range of proficiency rates: likely, some schools are underperforming while others are doing exceptionally well. ALSO: the number of students could skew aggregates. There could be a large school with a low rate that is dragging aggregates, or smaller schools pulling those up. 
